decoder transformer层的linear1层（dtl1）替换为近似矩阵乘法, 32\*32\*64, 64\*128, 32\*32\*128

In [9]:
import numpy as np
import os
import matmul as mm
import math_util as mu
# import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [10]:
# method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_EXACT
method = METHOD_SCALAR_QUANTIZE
nbits = 4 # 量化比特数

In [11]:
linear_name = 'dtl1'
feedback_bits = 256
ncodebooks = 32 # max:64
ncentroids = 16
if method == METHOD_MITHRAL:
    ncentroids = 16
train_sam_num = 3000 # 训练集样本数
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0
test_sam_num = 1000 # 测试集样本数
batch_size = 32

In [12]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= ''
    featurepath_train= ''
    data_to_fcpath_test = ''
    featurepath_test = ''
elif host_name == 'jm-System-Product-Name':
    dir_train = os.path.join('/data/hdr/transformer_data/joined', 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join('/data/hdr/transformer_data/joined', 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'dx_linear1in_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'dx_linear1_y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'dx_linear1out_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'dx_linear1in_test_f%i_sam%i.npy' % (feedback_bits, test_sam_num)
    featurepath_test = 'dx_linear1out_test_f%i_sam%i.npy' % (feedback_bits, test_sam_num)
    # y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dirs")


weightpath = 'dx_linear1_w_f256.npy'
biaspath = 'dx_linear1_b_f256.npy'
dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, linear_name)
try:
    os.mkdir(dir_result)
except FileExistsError:
    pass 


In [13]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method], nbits=nbits)


running method:  ScalarQuantize


In [14]:
# type(est3)

In [15]:
x_test = np.load(dir_test+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [16]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1]) #AMM字典模式需要复原y大小
print("y_out_last_re.shape: ", y_out_last_re.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
if method == METHOD_EXACT:
    train_sam_num = 0 # 训练集样本数
if method == METHOD_SCALAR_QUANTIZE:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_nbits%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, nbits)), y_out_last_re.astype(np.float32))
else:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, ncodebooks, ncentroids)), y_out_last_re)

[[ 0.7508716   0.23218285  0.26854654 ...  0.16047566 -0.6039159
   1.08534225]
 [ 0.37683328 -0.04204876 -0.28489902 ... -0.14682564 -0.01372764
   1.18594862]
 [-0.38729796  0.02146716 -0.2457492  ...  0.05414198  0.09685082
  -0.54608132]
 ...
 [ 1.17834102  0.29506239  1.20875909 ...  0.08226029 -0.05755875
   0.73207079]
 [ 0.82730373  0.24206681  0.42550909 ...  0.07232418 -0.391833
   1.18997596]
 [ 1.06481685  0.36330683  1.35948423 ...  0.04126037 -0.138184
   0.79988119]]
y_out_last.shape:  (1024000, 128)
y_out_last_re.shape:  (1000, 32, 32, 128)
